In [1]:
import requests

# Daily Recommended Calorie Calculator

In [2]:
def calories_macros_recommended(user_age,user_gender,user_height,user_weight,user_activity_level,user_goal):

    #REE : the number of calories it takes to run your body without movement

    # user inputs needed to calculate REE
    #user_age,user_gender,user_height,user_weight
    # REE Recommended
    # 10 x weight (kg) + 6.25 x height (cm) – 5 x age (y) + 5 = REE

    if user_gender == "Men":
        user_ree = 10*user_weight + 6.25*user_height - 5*user_age + 5
    user_ree = 10*user_weight + 6.25*user_height - 5*user_age - 161 #user is a woman
    
    
    # TDEE : the number of calories your particular body burns in a day
    
    #activity levels : 
    # Sedentary
    # Just everyday activities like a bit of walking, a couple of flights of stairs, eating, talking, etc. (REE X 1.2)
    # Light activity
    # Any activity that burns an additional 200-400 calories for females or 250-500 calories for males. (REE x 1.375)
    # Moderate activity
    # Any activity that burns an additional 400-650 calories for females or 500-800 calories for males. (REE x 1.55)
    # Very Active
    # Any activity that burns an additional 650+ calories for females or 800+ calories for males. (REE x 1.725) 

    activity_level_factor = {
        'Sedentary': 1.2,
        'Light activity': 1.375,
        'Moderate activity': 1.55,
        'Very Active': 1.725   
    }

    #user TDEE
    user_tdee = user_ree * activity_level_factor[f'{user_activity_level}']
    
    #User's goal: lose, maintain or gain weight

    goals_factor = {
        "Weight loss": 0.8,
        "Maintain": 1,
        "Weight gain": 1.1
    }

    user_final_tdee = user_tdee * goals_factor[f'{user_goal}']
    #Calculate the recommended protein / fat / carbs intake based on the calories
    # 1 g protein = 4 Calories
    # 1 g Carbohydrate = 4 Calories
    # 1 g fat = 9 Calories

    # Proteins :  Most people can use a more balanced approach and .825 g protein per pound
    weight_lbs = user_weight * 2.2 
    daily_protein_intake = weight_lbs * 0.825
    daily_protein_cals = daily_protein_intake * 4

    # Fats : Most research (and again, there are a lot of opinions) supports that 20%-30% of overall TDEE calories 
    # should come from the fat macro. Let’s use 25% and a middle-of-the-road starting point. 
    daily_fat_cals = user_tdee * 0.25
    daily_fat_intake = daily_fat_cals / 9

    # Carbs : We allocate the rest of our calories (originally calculated from our TDEE) to carbohydrates.
    daily_carb_cals = user_tdee - daily_protein_cals - daily_fat_cals
    daily_carb_intake = daily_carb_cals / 4
    calories_macros={}
    calories_macros['user_tdee']=user_tdee
    calories_macros['daily_protein_cals']=daily_protein_cals
    calories_macros['daily_fat_cals']=daily_fat_cals
    calories_macros['daily_carb_cals']=daily_carb_cals
    return calories_macros

In [3]:
calories_macros_recommended(20,'female',153,50,'Moderate activity',"Weight gain")

{'user_tdee': 1852.6375,
 'daily_protein_cals': 363.0,
 'daily_fat_cals': 463.159375,
 'daily_carb_cals': 1026.478125}

# Food Macro Calculator For Already Eaten Foods

In [4]:
def calculator(foods):
    cals = 0
    proteins = 0
    fats = 0 
    carbs = 0
    fibers = 0
    app_id="4e07f131"
    app_key="6f9bb3ba56ff2c9786adc5ade08cf1d2"
    for food in foods:
        
        url = f'https://api.edamam.com/api/food-database/v2/parser?app_id={app_id}&app_key={app_key}%09&ingr={food}&nutrition-type=cooking'
        r = requests.get(url).json()
        
        cal = r['hints'][0]['food']['nutrients']['ENERC_KCAL']
        protein = r['hints'][0]['food']['nutrients']['PROCNT']
        fat = r['hints'][0]['food']['nutrients']['FAT']
        carb = r['hints'][0]['food']['nutrients']['CHOCDF']
        fiber = r['hints'][0]['food']['nutrients'].get('FIBTG',0)
        cals += cal
        proteins += protein
        fats += fat 
        carbs += carb
        fibers += fiber
    return cals, proteins, fats, carbs, fibers

In [5]:
calculator(["sushi"])

(126.96415332408688,
 4.177344184499013,
 2.7546050573957666,
 20.90142982840552,
 0.7649823812494226)

# Food Recipe API

## inputs: food available, calories remaining

In [6]:
def food_recipe(food_choice,calories):

    recipe_key = '1c4f674c'
    recipe_id = 'f4063bcf2c650488f044cb2c2747fea3'
    calories = "400-500"
    food_choice = "chickpeas"

    url_2 = f"https://api.edamam.com/api/recipes/v2?type=public&q={food_choice}&app_id={recipe_key}&app_key={recipe_id}&calories={calories}"
    r_2 = requests.get(url_2).json()


    recipe_titles = r_2['hits']
    recipe=[]
    for dicts in recipe_titles:
        recipe.append(dicts['recipe']['label'])
    return recipe

In [7]:
food_recipe('pumpkin','500-600')

['Roasted Chickpeas',
 'Candied Spiced Chickpeas recipes',
 'Achiote Chickpeas',
 'Tamarind chickpeas',
 'Rigatoni with Roasted Broccoli and Chickpeas',
 'Harissa Quinoa With Shrimp and Chickpeas Recipe',
 'Butter chickpeas',
 'Tagliatelle with Chickpeas',
 'Spiced Roasted Chickpeas recipes',
 'Peppery Chickpeas recipes',
 'Couscous with Clementines, Chickpeas, Olives, and Dates recipes',
 'Butternut Squash Soup with Avocado & Chickpeas',
 'Aloo Gobi with Chickpeas recipes',
 'Radicchio Salad with Green Olives, Chickpeas, and Parmesan recipes',
 'Recipe: Braised Coconut Spinach & Chickpeas with Lemon',
 'Spiced Chickpeas and Greens',
 'Roasted Patty Pan Squash and Herbed Chickpeas',
 'Fried Chickpeas with Sage and Parmesan',
 'Cheesy Roasted Chickpeas (Chickpea Cheetos) recipes',
 'Pasta e Ceci (Pasta with Chickpeas)']